In [1]:
using Optim
using StatsFuns
using DataFrames

# inverse beta distribution function を行列に対応するように拡張
import StatsFuns.betainvcdf
betainvcdf(alpha::Number, beta::Number, x::Array) = reshape([betainvcdf(alpha, beta, i) for i in reshape(x, 1, size(x, 1)*size(x, 2)) ], size(x, 1), size(x, 2))

# maxをArray{String, 1}に対応するように拡張
# しかしArray型で入っているのでAnyに対応させる
import Base.max
max(number::Real, comparison::Any) = [max(number, parse(Int, i)) for i in comparison] 

# normpdfを配列に拡張
import StatsFuns.normpdf
normpdf(array::Array{Float64, 2}) = reshape([normpdf(i) for i in reshape(array, 1, size(array, 1)*size(array, 2))], size(array, 1), size(array, 2))

# parameterの初期値を作成
# inivalueは260こ
# learning_paramsは13こ
iii = 1 #kokokaeruiii;
ini = DataFrame(randn(260, 100))
learn = DataFrame(randn(13, 100))
writetable(*("inivalue_", "$iii", ".txt"), ini)
writetable(*("learning_params", "$iii", ".txt"), learn)

In [49]:
# define T as num of period
T = 14
bandwidth = 0.05
N_sim = 100
n_eval = 0
I = 4
N_cand = 4
# load iii.txt -ASCII;
# load jjj.txt -ASCII;

# dataのロード
data = readtable("data.csv", header = false)
data = convert(Array, data)

# initial valueのロード
iii = 1
jjj = 1
kkk = 1

inivalue = readtable(*("inivalue_", "$iii", ".txt"))
inivalue = inivalue[:, jjj]

learning_params = readtable(*("learning_params", "$iii", ".txt"))
learning_params = learning_params[:, jjj]

parameter = vcat(1000000000, 0, inivalue, learning_params)
writetable(*("parameter_", "$iii", "_", "$jjj", "_", "$kkk", ".txt"), DataFrame(X = parameter))


#　ここから本題
D　=　data[:,45] .> 100 # Remove municipality with small populaion <100
# 行削除はどうするのか、そもそもいるのか
# data[D, :] = [];
data = data[D, :]

# Construct X : Regressor for individual characteristics
X = [1 1 1 1 1];
dFX = ones(size(data, 1))
Race = [1 0 0; 0 1 0; 0 0 1];
Educ = 0.1*[16;14;12;9];
Incm = log([20000;35000;72500;120000]);

# (white,black,otherasian+indian+other)
dFXRace = [data[:,121] data[:,124] data[:,122] + data[:,123] + data[:,125]];

dFXEduc = [data[:,126:128] (1- sum(data[:,126:128], 2))];  # (overba,underba,hs,other)
dFXIncm = [sum(data[:,129:132], 2) sum(data[:,133:136], 2) sum(data[:,137:140], 2) sum(data[:,141:144], 2)];

# dFXIncm=data(:,129:144); % (income 16 category:
# -10K,15K,20K,25K,30K,35K,40,45,50,60,75,100,125,150,200,200+)
# ここは精査する必要がありそう。
for r in 1:3
    for e in 1:4
        for i in 1:4
            X = [X; reshape([Race[r,1:3] ; Educ[e] ; Incm[i]], 1, 5)];
            dFX = [dFX dFXRace[:,r].*dFXEduc[:,e].*dFXIncm[:,i]];
        end
    end
end

del = trues(size(X, 1))
del[1] = false
X = X[del, :]
del2 = trues(size(dFX, 2))
del2[1] = false
dFX = dFX[:, del2]

N_dFX = size(dFX,2);
N_muni = size(dFX,1);      # (number of municipalities)
# data3,4,5,6はまさかのstring型
# (votes:  clark, dean, edwards, kerry)municipalityごとに、いない奴は0になるようにしてる
Votes = [max(0,data[:,3]) max(0,data[:,4]) max(0,data[:,5]) max(0,data[:,6])]; 
VTotal = data[:,117];      # (total number of votes)
RDemHat = data[:,148];     # (registred number of democrats predicted)
PopTot = data[:,45];       # (total population)
Open = data[:,118];        # open
MOpen = data[:,119];       # modified open
VOther = VTotal - sum(Votes,2);  #Votes of Penna & Sharpton etc.
RegDem = RDemHat./PopTot;  # (fraction of registered democrats in population)

# make Cand
# 何で18列目から21列めまでを開けてる？
Cand = zeros(35,27);       

                         # Candidates whose name is on ballot (column 1;4)
                         # Column 5 corresponds to ichiban maeno parameter
                         # noichi (for T_ij), Col6 to ichiban saigono parameter.
                         # Same for Columns 7 and 8 (for E[qi|omega]).
                         # Column 9: Who's in the race.
                         # Column 10: 1 if Before (and including) super tuesday
                         # Column 11:　？
                         # Column 12:　？
                         # Column 13: Date t (1,2,...14). 6 correspond to
                         # super Tues.
                         # Column 14: Municipality Number Start
                         # Column 15: Municipality Number End;
                         # 下のE[qi|omega_piv]-E[qi|omega_piv]って0では？
                         # Column 16: Atamadashi for
                         # E[qi|omega_piv]-E[qi|omega_piv]
                         # Column 17: Owaridashi for
                         # E[qi|omega_piv]-E[qi|omega_piv]
                         # the remains is explained below

j = 0
N_T_ij = [0;1;3;6];        # The number of T_ij that we need when the number of candidates are 1,2,3,4.
for i in 1:size(data, 1)
    if j == data[i,149] - 1
        Cand[j+1,1:4] = squeeze(ones(1,4), 1)+min(data[i,3:6], squeeze(zeros(1,4), 1));
        if j == 0
            Cand[j+1,5] = 0;
            Cand[j+1,6] = 0;
            Cand[j+1,11] = 0;
        else
            Cand[j+1,5] = Cand[j,6] + (sum(Cand[j+1,1:4])>1);
            Cand[j+1,6] = Cand[j+1,5] + N_T_ij[convert(Int64, sum(Cand[j+1,1:4])),1] - 1+(sum(Cand[j+1,1:4])==1);
        end

        # 1 if before or on super Tues.

        Cand[j+1,10] = (data[i,120] - 92<=0);
        Cand[j+1,7] = sum(sum(Cand[1:j+1,1:4]))-sum(Cand[j+1,1:4])+1;
        Cand[j+1,8] = sum(sum(Cand[1:j+1,1:4]));

        Cand[j+1,17] =  sum(sum(Cand[1:j+1,1:4].*(Cand[1:j+1,10]*ones(1,4))));
        Cand[j+1,16] = Cand[j+1,17]-sum(Cand[j+1,1:4]*Cand[j+1,10])+1;
        Cand[j+1,16:17] = Cand[j+1,16:17]*Cand[j+1,10];

        # Baai-wake: [Clark,Dean,Edawards]
        # Type 0: [000] Type 1:[111] Type2:[101] Type3: [011] Type4: [010]
        if Cand[j+1,1:3] == [0,0,0]
            Cand[j+1,9] = 0;
        elseif Cand[j+1,1:3] == [1, 1, 1]
            Cand[j+1,9] = 1;
        elseif Cand[j+1,1:3] == [1, 0, 1]
            Cand[j+1,9] = 2;
        elseif Cand[j+1,1:3] == [0, 1, 1]
            Cand[j+1,9] = 3;
        else
            Cand[j+1,9] = 4;
        end

        #Atamadashi for params
        if j > 0
            a = sum((Cand[1:j,6] - Cand[1:j,5]+1).*Cand[1:j,10],1)+Cand[j+1,10]*(sum(Cand[j+1,1:4]) > 1)
            Cand[j+1,11] = a[1];
            b = sum((Cand[1:j+1,6] - Cand[1:j+1,5]+1).*Cand[1:j+1,10],1)
            Cand[j+1,12] = b[1];
        end
        
        Cand[j+1,13] = data[i,120];
        Cand[j+1,14] = i;
         j = j+1;
        
    end


    Cand[:,11] = Cand[:,10].*Cand[:,11];      # Atamadashi ignoring post-super
    Cand[:,12] = Cand[:,10].*Cand[:,12];      # Tues states
    Cand[1:end-1,15] = Cand[2:end,14]-1;
    Cand[end,15] = i;

end

iij = unique(Cand[:,13]);
for i in 1:size(unique(Cand[:,13]),1)
    for j in 1:size(Cand[:,13],1)
        if Cand[j,13] == iij[i]
            Cand[j,13] = i;
        end
    end
end

w = convert(Int64, maximum(Cand[:,13]))
NNCan = ones(w, 1)
A_Exi = ones(w, 2)
PatternCandall = ones(w, 4)
NNCan[1,1] = 4;
A_Exi[1,:] = [1 4];
PatternCandall[1,:] = [1 1 1 1];

for S in 2:w
    # of candidate on date S
    Temp = Cand;
    # 行削除はどうする
    # Temp[(Temp[:,13] .!= S), :]=[];
    Temp = Temp[Temp[:,13] .== S, :]
    NNCan[S,1] = sum(sum(Temp[:,1:4],1) .> 0);
    PatternCandall[S,1:4] = (sum(Temp[:,1:4]) > 0);
    # Atamadashi for ExiOmega
    A_Exi[S,:] = [A_Exi[S-1,2]+1 (A_Exi[S-1,2] + NNCan[S,1])];
end

for S = 1:size(Cand,1)
    SS = convert(Int64, Cand[S,13]);
    Cand[S,22:23] = A_Exi[SS,:];
    Cand[S,24:27] = PatternCandall[SS,:];
end


# parameter description
# Xi|Omg (118x1)     : For each state, we have at most 4 values (total 118)
# Xi|OmgPiv (56x1)   : For each state, we have at most 3 values (total  56)
# Tij (105x1)        : For each state, we have at most 6 values (total 105)
#  Set to zero if after Super Tuesday.
# Cx    (sizeXk x1)  : Cost function parameter.  Number of Xk
# Cz    (3 x 1)      : Cost function parameter related to other elections
# vk    (sizeXk x4)  : Preference paramether.  Number of Xk x Number of Cand
# FAlph (2x1)        : Distribution of alpha (beta dist), 2x1
# Sig_xsi (1x1)      : variance of Xsi
# DeltaO  (1x1)       : Increase of eligible voters for open election
# DeltaMO (1X1)       : Increase of eligible voters for modified opene elec.

# load inivalue.txt inivalue -ASCII
# load indicator.txt indicator -ASCII;
# load inival.txt inival -ASCII

DATA = data;
srand(10);

# add XiOmg
# load the simulated XiOmg data
# reshape the data to [T, N_cand, N_sim]
SimXsi = randn(N_muni, N_cand, N_sim);
SimAlp = rand(N_muni, N_sim);



In [67]:
function new_loglike(param::Array{Float64,1})
    
    # これなんだかわからないのでコメントアウト
    #if i_bayes == 1
        #param = param
    #end
    
    FAlph = Array(Float64, 2, 1)
    # setting parameters
    param[2] = -1
    C0 = 0
    # Raceの変更に伴い、Cxも4つの要素にする必要がある。
    q = 4
    Cx = param[1:q]
    Cz = -abs(param[q+1:q+3])
    FAlph[1,1] = abs(param[q+4])
    FAlph[2,1] = abs(param[q+5])
    param[q+4:q+5] = FAlph[1:2,1]
    Sig_xsi  = max(0.5, abs(param[q+6]))
    param[q+6] = Sig_xsi
    DeltaO  = 0.6891
    DeltaMO = 0.5366
    # Raceをいじった関係でこのvkを4*4 = 16この要素にしなくちゃいけない
    vk = param[12:27]
    composite = param[75:149]
    Tij = abs(param[150:260])
    
    # new parameters
    rho_eta = abs(param[261])
    rho_chi = param[262:265]
    mu_chi = param[266:269]
    chi = param[270:273]
    
    
    # making log likelihood

    # simulated values
    N_T_ij = [0;1;3;6]
    # 上で拡張した関数を使用
    Alpha = betainvcdf(FAlph[1,1],FAlph[2,1], SimAlp)
    Xsi = Sig_xsi*SimXsi

    # calculating utilities
    # Base Utility for Sincere, [# of categories X # of candidates]
    # VSinは48×4
    VSin = X[:,2:end]*[vk[1:4] vk[5:8] vk[9:12] vk[13:16]]-C0-X[:, 2:end]*Cx*ones(1,4)
    # Base Utility for Strategic, [# of categories X # of candidates]
    # VStrも48×4
    VStr = X[:, 2:end]*[vk[1:4] vk[5:8] vk[9:12] vk[13:16]]

    #Eligible voters accounting for Open and Modified Open
    DELTA = DeltaO*Open+DeltaMO*MOpen
    RTOT = RDemHat.*(1+DELTA)-VOther

    # store signals in advance
    signals = randn(4,T) * sqrt(1/rho_eta)
    for i in 1:4
        signals[i, :] = signals[i, :] + chi[i, 1]
    end

    loglik_s = zeros(size(Cand,1),1)

    # Candをdateについての昇順に変える
    # sort(Cand,13)
    sortrows(Cand, by = x->(x[13]))
    
    col9 = convert(Array{Int64,1},Cand[:, 9])
    col10 = convert(Array{Int64,1},Cand[:, 10])
    col11 = convert(Array{Int64,1},Cand[:, 11])
    col12 = convert(Array{Int64,1},Cand[:, 12])
    col13 = convert(Array{Int64,1},Cand[:, 13])
    col14 = convert(Array{Int64,1},Cand[:, 14])
    col15 = convert(Array{Int64,1},Cand[:, 15])
    col16 = convert(Array{Int64,1},Cand[:, 16])
    col17 = convert(Array{Int64,1},Cand[:, 17])
    
    for S in 1:size(Cand,1)
        if col15[S] - col14[S] < 21 || S == 30 || S == 34 #Excluding Utah, Wisconsin, and small
        else
            N_candS = sum(Cand[S,1:4] .!= 0.0)
            M = col15[S] - col14[S] + 1    # Number of municipalities in State S
            
            if col11[S] == 0
                T_s = zeros(N_T_ij[N_candS,1],1)
                COMPOSITE = zeros(N_candS,1)
            else
                T_s = Tij[col11[S]:col12[S],1]
                COMPOSITE = composite[col16[S]:col17[S],1]
            end

            # VSTR_s = []
            Dropped_s = find(Cand[S,1:4] .== 0.0)            # index of candidates withdrawn.
            Rem = Cand[S,1:4] .!= 0.0
            print(Rem)
            Candidate_s = find(Cand[S,1:4] .== 1.0)        # index of cadidate
            Senate_s = DATA[col14[S], 27]*Cz[1,1]     # Cost from Senate elections
            Governer_s = DATA[col14[S], 29]*Cz[3,1]    # Cost from GOvernor elections
            dFX_s = dFX[col14[S]:col15[S], :]
            date = col13[S]                          # election date (1 ~ 14)

            # というか列の削除をする必要があるのか
            temp = Cand[S,1:4]
            PatternCand_d = Cand[S, 24:27]
            # temp[:, find(Cand[S,24:27] .== 0)] = []
            # PatternCand_d[:, find(Cand[S,24:27] .== 0)] = []
            temp = temp[Cand[S,24:27] .!= 0.0]
            PatternCand_d = PatternCand_d[Cand[S,24:27] .!= 0.0]

            Alpha_s = reshape(Alpha[col14[S]:col15[S], :, :], length(col14[S]:col15[S]), 100)
            Xsi_s = Xsi[col14[S]:col15[S], :, :]

            # そもそも削除する必要があるか
            VSin_s = VSin
            # VSin_s[:, Dropped_s] = []
            VSin_s = VSin_s[:, Rem]
            VStr_s = VStr
            # println(size(VStr_s, 2))
            # VStr_s[:, Dropped_s] = []
            VStr_s = VStr_s[:, Rem]
            print(size(VStr_s, 2))

            # make XiOmg_s whose size is [1, num of candidates in the state]
            # take necessary parameters
            rho_chi_s = rho_chi[Candidate_s, 1]
            mu_chi_s = mu_chi[Candidate_s, 1]
            chi_s = chi[Candidate_s, 1]
            # calculating XiOmg_s
            # signals_s = signals[:, date]
            
            # cumsum は1次元配列にしか使えないので注意ダメだったらsqueeze
            cum_signals = cumsum(signals, 2)
            upper = rho_chi_s + mu_chi_s + rho_eta * cum_signals[Candidate_s, date]
            under = rho_chi_s + date * rho_eta
            XiOmg_s = upper ./ under

            VSin_s = VSin_s + ones(size(VSin_s,1), 1)*XiOmg_s' -Senate_s-Governer_s
            
            # col9は0~4、4は？

            print(col9[S])
            if col9[S] == 1 && col10[S] == 1
                VSTR_s = ones(48, 4)
                Composite = ones(size(X,1),1)*COMPOSITE'
                VSTR_s[:,1] = T_s[1,1]*(VStr_s[:,1]-VStr_s[:,2])+T_s[2,1]*(VStr_s[:,1]-VStr_s[:,3])
                +T_s[3,1]*(VStr_s[:,1]-VStr_s[:,4])+Composite[:,1]
                VSTR_s[:,2] = T_s[1,1]*(VStr_s[:,2]-VStr_s[:,1])+T_s[4,1]*(VStr_s[:,2]-VStr_s[:,3])
                +T_s[5,1]*(VStr_s[:,2]-VStr_s[:,4])+Composite[:,2]
                VSTR_s[:,3] = T_s[2,1]*(VStr_s[:,3]-VStr_s[:,1])+T_s[4,1]*(VStr_s[:,3]-VStr_s[:,2])
                +T_s[6,1]*(VStr_s[:,3]-VStr_s[:,4])+Composite[:,3]
                VSTR_s[:,4] = T_s[3,1]*(VStr_s[:,4]-VStr_s[:,1])+T_s[5,1]*(VStr_s[:,4]-VStr_s[:,2])
                +T_s[6,1]*(VStr_s[:,4]-VStr_s[:,3])+Composite[:,4]

            # ここに問題がある?
            elseif (col9[S] == 2 || col9[S] == 3 || col9[S] == 4) && col10[S] == 1
                VSTR_s = ones(48, 3)
                Composite = ones(size(X,1),1)*COMPOSITE'
                VSTR_s[:,1] = T_s[1,1]*(VStr_s[:,1]-VStr_s[:,2])+T_s[2,1]*(VStr_s[:,1]-VStr_s[:,3])
                +Composite[:,1]
                VSTR_s[:,2] = T_s[1,1]*(VStr_s[:,2]-VStr_s[:,1])+T_s[3,1]*(VStr_s[:,2]-VStr_s[:,3])
                +Composite[:,2]
                VSTR_s[:,3] = T_s[2,1]*(VStr_s[:,3]-VStr_s[:,1])+T_s[3,1]*(VStr_s[:,3]-VStr_s[:,2])
                +Composite[:,3]


            elseif col10[S] == 0  # after super tuesday
                # VSTR_s=VSin_s+C0+X(:,2:end)*Cx(1:3)*ones(1,N_candS) ?
                VSTR_s = VSin_s + 2*(Senate_s+Governer_s)
            end

            print(N_candS)
            println(size(VSTR_s, 2))
            # Utiltiy of Strategic with no house elections
            VSTR_s = VSTR_s - C0 - X[:,2:end]*Cx*ones(1,N_candS) - Senate_s - Governer_s

            VSTR_ss = zeros(N_sim,N_candS)
            VSIN_ss = zeros(N_sim,N_candS)

            # eligible voters
            RTot_s = RTOT[col14[S]:col15[S], :]
            RTot_s = max(RTot_s, sum(Votes[col14[S]:col15[S], :], 2))
            Votes_s = Votes[col14[S]:col15[S], :]./(RTot_s*ones(1,4)) #vote share data
            # 行削除
            # Votes_s[:, Dropped_s] = []
            Votes_s = Votes_s[:, Rem]
            loglik_m = zeros(M,1)

            if col10[S] == 1

                for m in 1:M
                    VSin_s = VSin_s - Cz[2,1]*DATA[col14[S] + m - 1, 28]
                    VSTR_s = VSTR_s - Cz[2,1]*DATA[col14[S] + m - 1, 28]

                    for sim in 1:N_sim

                        nakami = max(min(VSin_s + ones(N_dFX,1)*reshape(Xsi_s[m,1:N_candS, sim], 1, N_candS), 200), -200)
                        nakami2 = max(min(VSTR_s + ones(N_dFX,1)*reshape(Xsi_s[m,1:N_candS,sim], 1, N_candS), 200), -200)

                        eVSIN_ss = exp(nakami)./ (1+sum(exp(nakami),2)*ones(1,N_candS))
                        eVSTR_ss = exp(nakami2)./ (1+sum(exp(nakami2),2)*ones(1,N_candS))

                        VSTR_ss[sim, :] = reshape(dFX_s[m, :],1, 48)*eVSTR_ss
                        VSIN_ss[sim, :] = reshape(dFX_s[m, :], 1, 48)*eVSIN_ss
                    end

                    Alp_ss = Alpha_s[m, :]
                    VSHARE = VSTR_ss.*(Alp_ss*ones(1,N_candS)) + VSIN_ss.*(1-Alp_ss*ones(1,N_candS))
                    # pdfはStatsFunsのnorrmpdfを使用
                    loglik_m[m,1] = log(sum(prod(normpdf((ones(N_sim,1)*reshape(Votes_s[m,:], 1, N_candS) - VSHARE)/bandwidth),2),1)/N_sim)[1]
                end
        
                loglik_s[S,1] = sum(loglik_m)

            elseif col10[S] == 0

                for m in 1:M
                    VSin_s = VSin_s-Cz[2,1]*DATA[col14[S]+m-1, 28]
                    AST = 1./(1+exp(C0+X[:,2:end]*Cx*ones(1,N_candS) + Cz[2,1]*DATA[col14[S]+m-1, 28]+Senate_s+Governer_s))
                    # AST: Turnout of strategic voters after super tuesday
                    
                    for sim in 1:N_sim

                        nakami = max(min(VSin_s + ones(N_dFX,1)*reshape(Xsi_s[m, 1:N_candS,sim], 1, N_candS), 200), -200)
                        eVSIN_ss = exp(nakami)./ (1 + sum(exp(nakami),2)*ones(1,N_candS))
                        VSIN_ss[sim, :] = reshape(dFX_s[m, :], 1, 48) * eVSIN_ss
                    
                    end
        
                    VSHARE = VSIN_ss
                    # pdfはStatsFunsのnorrmpdfを使用
                    loglik_m[m,1] = log(sum(prod(normpdf((ones(N_sim,1)*reshape(Votes_s[m,:], 1, N_candS) - VSHARE)/bandwidth),2),1)/N_sim)[1]
                end
                loglik_s[S,1] = sum(loglik_m)
            end
        end
        
        if S == 10
            S = S
        end
    end

    loglik = sum(loglik_s)
    return loglik = -loglik

end

new_loglike (generic function with 1 method)

In [68]:
new_loglike(parameter)

Bool[false,false,false,true]1011
Bool[false,false,false,true]1011
Bool[true,true,true,true]4144
Bool[true,true,true,true]4144
Bool[true,true,true,true]4144
Bool[true,true,true,true]4144
Bool[true,true,true,true]4144
Bool[true,true,true,true]4144
Bool[true,true,true,true]4144
Bool[true,true,true,true]4144
Bool[true,true,true,true]4144
Bool[true,true,true,true]4144
Bool[true,true,true,true]4144
Bool[true,true,true,true]4144
Bool[true,false,true,true]3233
Bool[false,true,true,true]3333
Bool[true,true,true,true]4144
Bool[true,true,true,true]4144
Bool[true,true,true,true]4144
Bool[true,true,true,true]4144
Bool[false,false,false,true]1011
Bool[false,true,true,true]3333
Bool[true,true,true,true]4144
Bool[false,true,false,true]2422
Bool[true,true,true,true]4144
Bool[true,true,true,true]4144
Bool[true,true,true,true]4144
Bool[true,true,true,true]4144
Bool[true,true,true,true]4144
Bool[true,true,true,true]4144


428885.6981348013

In [7]:
param = parameter
FAlph = Array(Float64, 2, 1)
# setting parameters
param[2] = -1
C0 = 0
# Raceの変更に伴い、Cxも4つの要素にする必要がある。
q = 4
Cx = param[1:q]
Cz = -abs(param[q+1:q+3])
FAlph[1,1] = abs(param[q+4])
FAlph[2,1] = abs(param[q+5])
param[q+4:q+5] = FAlph[1:2,1]
Sig_xsi  = max(0.5, abs(param[q+6]))
param[q+6] = Sig_xsi
DeltaO  = 0.6891
DeltaMO = 0.5366
# Raceをいじった関係でこのvkを4*4 = 16この要素にしなくちゃいけない
vk = param[12:27]
composite = param[75:149]
Tij = abs(param[150:260])

# new parameters
rho_eta = abs(param[261])
rho_chi = param[262:265]
mu_chi = param[266:269]
chi = param[270:273]


# making log likelihood

# simulated values
N_T_ij = [0;1;3;6]
# 上で拡張した関数を使用
# ここまではきてる。
Alpha = betainvcdf(FAlph[1,1],FAlph[2,1], SimAlp)
Xsi = Sig_xsi*SimXsi

# calculating utilities
# Base Utility for Sincere, [# of categories X # of candidates]
# VSinは48×4
VSin = X[:,2:end]*[vk[1:4] vk[5:8] vk[9:12] vk[13:16]]-C0-X[:, 2:end]*Cx*ones(1,4)
# Base Utility for Strategic, [# of categories X # of candidates]
VStr = X[:, 2:end]*[vk[1:4] vk[5:8] vk[9:12] vk[13:16]]
# ここまでは来てる

#Eligible voters accounting for Open and Modified Open
DELTA = DeltaO*Open+DeltaMO*MOpen
RTOT = RDemHat.*(1+DELTA)-VOther
# ここまでは来てる

# store signals in advance
signals = randn(4,T) * sqrt(1/rho_eta)
for i in 1:4
    signals[i, :] = signals[i, :] + chi[i, 1]
end
# ここまで来た

loglik_s = zeros(size(Cand,1),1)

# Candをdateについての昇順に変える
# sort(Cand,13)
sortrows(Cand, by = x->(x[13]))
# ここまで来た

col9 = convert(Array{Int64,1},Cand[:, 9])
col10 = convert(Array{Int64,1},Cand[:, 10])
col11 = convert(Array{Int64,1},Cand[:, 11])
col12 = convert(Array{Int64,1},Cand[:, 12])
col13 = convert(Array{Int64,1},Cand[:, 13])
col14 = convert(Array{Int64,1},Cand[:, 14])
col15 = convert(Array{Int64,1},Cand[:, 15])
col16 = convert(Array{Int64,1},Cand[:, 16])
col17 = convert(Array{Int64,1},Cand[:, 17])

35-element Array{Int64,1}:
  0
  4
  0
  8
 12
 16
  0
 20
  0
  0
  0
  0
 24
  ⋮
  0
  0
 52
  0
 56
  0
 59
 63
 67
  0
 71
 75

In [58]:
S = 1
Alpha_s = reshape(Alpha[col14[S]:col15[S], :, :], length(col14[S]:col15[S]), 100)

67×100 Array{Float64,2}:
 0.998222  0.983932  0.698784   0.999187   …  0.941881   0.807749  0.627685 
 0.999506  0.401013  0.787592   0.317371      0.44039    0.512903  0.734809 
 0.948137  0.948984  0.481319   0.95635       0.995569   0.747213  0.52199  
 0.718228  0.654588  0.458276   0.995444      0.995369   0.995396  0.948133 
 0.975255  0.776017  0.0925888  0.530565      0.761962   0.991584  0.180841 
 0.77737   0.992653  0.338955   0.75591    …  0.953501   0.999781  0.637415 
 0.526568  0.795198  0.965824   0.0550064     0.820482   0.788698  0.970684 
 0.978655  0.986694  0.948513   0.898626      0.973727   0.96279   0.622064 
 0.421798  0.281364  0.80025    0.409202      0.955235   0.97537   0.976645 
 0.944237  0.998122  0.960822   0.365452      0.0966125  0.522063  0.467762 
 0.926669  0.901495  0.342527   0.907921   …  0.881397   0.382668  0.876517 
 0.490783  0.99397   0.315309   0.744182      0.823549   0.408197  0.341706 
 0.999115  0.765562  0.582221   0.980619      0.946

In [62]:
m = 1
Alp_ss = Alpha_s[m, :]*ones(1,4)

100×4 Array{Float64,2}:
 0.998222   0.998222   0.998222   0.998222 
 0.983932   0.983932   0.983932   0.983932 
 0.698784   0.698784   0.698784   0.698784 
 0.999187   0.999187   0.999187   0.999187 
 0.760276   0.760276   0.760276   0.760276 
 0.999984   0.999984   0.999984   0.999984 
 0.969051   0.969051   0.969051   0.969051 
 0.0256385  0.0256385  0.0256385  0.0256385
 0.321757   0.321757   0.321757   0.321757 
 0.996109   0.996109   0.996109   0.996109 
 0.860174   0.860174   0.860174   0.860174 
 0.971866   0.971866   0.971866   0.971866 
 0.999447   0.999447   0.999447   0.999447 
 ⋮                                         
 0.998332   0.998332   0.998332   0.998332 
 0.997669   0.997669   0.997669   0.997669 
 0.875292   0.875292   0.875292   0.875292 
 0.782276   0.782276   0.782276   0.782276 
 0.8776     0.8776     0.8776     0.8776   
 0.885488   0.885488   0.885488   0.885488 
 0.87034    0.87034    0.87034    0.87034  
 0.978399   0.978399   0.978399   0.978399 
 0.70015

In [57]:
length(1:10)

10